In [1]:
import numpy as np
from sklearn.model_selection import KFold
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
from matplotlib.collections import LineCollection
import os
from utils.color_map import *

In [2]:
# load data
result_path = os.path.abspath('../results/Ach-ver16')
folder_name = 'voxel-integration-region-SRD-long-formal'
num_dim_file = 'list_list_num_dim.npy'
fly_selected = [0,1,2,3,4,5,6,7,8,9]
# parameters 
if_shuffle = False
thresh = np.arange(0,1,0.02)
channel = np.array([0,1,2])
num_channel_choice = len(channel)
dim_thresh = 25
if_save =  False
if if_save:
    figure_save_path = result_path + '/' + 'figures-for-ver16-final6'+'/manifold-SRD-long-formal--3-40'
    f = os.path.exists(figure_save_path)
    if not f:
        os.makedirs(figure_save_path)
cv_fold = 5
kf = KFold(n_splits=cv_fold,shuffle = True,random_state = 5)
time_range = range(0,43)# time range

# load data
file_folders = os.listdir(result_path)
print(file_folders)
num_fly = len(fly_selected)
yy = np.zeros((num_fly,num_channel_choice))
for i in range(num_fly):
    file_folder = file_folders[fly_selected[i]]
    the_path = result_path + '/' + file_folder + '/' + folder_name
    f = os.path.exists(the_path + '/' + num_dim_file)
    if not f:
        continue
    num_dim = np.load(the_path + '/' + num_dim_file)
    num_dim = np.squeeze(num_dim)
    if not if_shuffle:
        num_dim = num_dim[:,:,0]
    for j in range(num_channel_choice):
        a = np.where(np.squeeze(num_dim[j,:])>=dim_thresh)
        a = np.array(a[0])
        yy[i,j] = thresh[a[0]]

['20230417-fly2', '20230420-fly2', '20230420-fly3', '20230428-fly1', '20230507-fly1', '20230510-fly1', '20230510-fly2', '20230511-fly2', '20230511-fly3', '20230515-fly1', 'DEEPCAD', 'figures', 'figures-for-ver16-final6']


In [3]:
for ii in range(num_fly):
    file_folder = file_folders[fly_selected[ii]]
    the_path = result_path + '/' + file_folder + '/' + folder_name
    the_sub_path = os.listdir(the_path)
    print(the_sub_path)
    j_flag = 0
    for j in range(len(the_sub_path)):
        if 'channel' not in the_sub_path[j]:
            continue
        the_sub_path_full = the_path + '/' + the_sub_path[j] + '/' + 'No_Shuffle'
        sub_data_path = os.listdir(the_sub_path_full)
        for m in range(len(sub_data_path)):
            if str(yy[ii,j_flag])+'_' in sub_data_path[m] and '.npy' in sub_data_path[m] :
                the_final_path = the_sub_path_full + '/' + sub_data_path[m]
                data = np.load(the_final_path)
                stim_file_name = 'stim.npy'
                stim = np.load(the_sub_path_full + '/' + stim_file_name)
                print(np.shape(stim))
                num_trial = len(stim)
                list_train = []
                list_test = []
                for train, test in kf.split(np.linspace(0,num_trial-1,num_trial)):
                    list_train.append(train)
                    list_test.append(test)
                list_train = np.array(list_train)
                list_test = np.array(list_test)
                filename = sub_data_path[m]
                if data.ndim==2:
                    stim_flag = stim
                    data = data.reshape((int(np.size(data,0)/num_trial),num_trial,-1),order = 'F')
                    data = data[:,:,[0,1]]###
                    trial_flag = range(180)
                    num_tp = len(time_range)
                else:
                    number = int(filename[-5:-4])-1
                    flag = filename[0:4]
                    if flag == 'test':
                        stim_flag = stim[list_test[number,:]]
                        trial_flag = list_test[number,:]
                    else:
                        stim_flag = stim[list_train[number,:]]
                        trial_flag = list_train[number,:]
                    num_tp = len(time_range)
                #################################
                # plot-1 color-stim 
                color_list = ['royalblue','firebrick','darkorange']
                label = ['OCT','MCH','EA']
                plt.figure(figsize=(3,3))
                ax = plt.axes()
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                cnt = [0,0,0]
                ############
                # rearrange
                start = (np.squeeze(data[0,:,:])).copy()
                start = np.mean(start,0)
                for i in range(np.size(data,0)):
                    data[i,:,:] = data[i,:,:]-start
                a = data[5:13,stim_flag==1,:]
                a = np.mean(np.mean(a,0),0)
                b = data[5:13,stim_flag==2,:]
                b = np.mean(np.mean(b,0),0)
                c = data[5:13,stim_flag==3,:]
                c = np.mean(np.mean(c,0),0)
                if a[1]<b[1]:# OCT MCH opposite
                    data[:,:,1] = 0-data[:,:,1]
                if c[0]<b[0]:# EA MCH 
                    data[:,:,0] = 0-data[:,:,0]
                ####################
                for i in range(np.size(data,1)):
                    plt.plot(data[time_range,i,0],data[time_range,i,1],c = color_list[stim_flag[i]-1],label = label[stim_flag[i]-1],linewidth=1.5)
                    cnt[stim_flag[i]-1] = cnt[stim_flag[i]-1]+1
                    if cnt[0]==1 and cnt[1]==1 and cnt[2]==1:
                        plt.legend(loc = 4)
                # ax.set_xlim(np.min(data[:,:,0])-0.1*abs(np.min(data[:,:,0])), np.max(data[:,:,0])+0.1*abs(np.max(data[:,:,0])))
                # ax.set_ylim(np.min(data[:,:,1])-0.1*abs(np.min(data[:,:,1])), np.max(data[:,:,1])+0.1*abs(np.max(data[:,:,1])))
                ax.set_xlim(-11,10)
                ax.set_ylim(-10,10)
                if if_save:
                    mpl.rcParams['pdf.fonttype'] = 42
                    mpl.rcParams['ps.fonttype'] = 42
                    the_save_path = figure_save_path +'/'+file_folder +'_No_shuffle'
                    f = os.path.exists(the_save_path)
                    if not f:
                        os.makedirs(the_save_path)
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename +'_stim.pdf',dpi = 300,bbox_inches = 'tight')
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename +'_stim.png',dpi = 300,bbox_inches = 'tight')
                plt.close()
                ######################################
                # plot-2 color-trial
                color_list = plt.cm.coolwarm(np.linspace(0,1,num_trial))

                fig = plt.figure(figsize=(3,3))
                ax = plt.axes()
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                m = plt.subplot(1,1,1)
                cnt = [0,0,0]
                for i in range(np.size(data,1)):
                    plt.plot(data[time_range,i,0],data[time_range,i,1],c = color_list[trial_flag[i]],linewidth=1.5)
                # m.set_xlim(np.min(data[:,:,0])-0.1*abs(np.min(data[:,:,0])), np.max(data[:,:,0])+0.1*abs(np.max(data[:,:,0])))
                # m.set_ylim(np.min(data[:,:,1])-0.1*abs(np.min(data[:,:,1])), np.max(data[:,:,1])+0.1*abs(np.max(data[:,:,1])))
                ax.set_xlim(-11,10)
                ax.set_ylim(-10,10)
                fig.subplots_adjust(bottom=0.108)

                #colorbar
                l = 0.7
                b = 0
                w = 0.2
                h = 0.02 
                rect = [l,b,w,h] 
                cbar_ax = fig.add_axes(rect) 
                cmap1 = copy.copy(mpl.cm.coolwarm)
                norm1 = mpl.colors.Normalize(vmin=1, vmax=num_trial)
                im1 = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap1)
                cbar1 = plt.colorbar(
                    im1, orientation='horizontal',
                    ticks=np.linspace(1, num_trial, 2),
                    label='Trials',cax=cbar_ax)
                if if_save:
                    mpl.rcParams['pdf.fonttype'] = 42
                    mpl.rcParams['ps.fonttype'] = 42
                    the_save_path = figure_save_path +'/'+file_folder +'_No_shuffle'
                    f = os.path.exists(the_save_path)
                    if not f:
                        os.makedirs(the_save_path)
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename +'_trial.png',dpi = 300,bbox_inches = 'tight')
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename +'_trial.pdf',dpi = 300,bbox_inches = 'tight')
                plt.close()
                ########################################
                # plot-3 color-tp
                flag = 0
                for train, test in kf.split(np.linspace(0,num_trial-1,num_trial)):
                    if flag ==int(filename[-5])-1:
                        if filename[0:4]=='trai':
                            stim_train = train
                        else:
                            stim_train = test
                        break
                    flag=flag+1
                fig = plt.figure(figsize=(3,3))
                ax = plt.axes()
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                m = plt.subplot(1,1,1)
                data = data[time_range,:,:]
                data_mean = np.zeros((np.size(data,0),3,np.size(data,2)))
                for i in range(3):
                    data_mean[:,i,:] = np.mean(data[:,stim[stim_train]==i+1,:],1)
                    x1 = np.linspace(0,1,num_tp)
                    y1 = data_mean[:,i,0]
                    x2 = np.linspace(0,1,num_tp)
                    y2 = data_mean[:,i,1]
                    ps = np.stack((y1,y2), axis=1)
                    segments = np.stack((ps[:-1], ps[1:]), axis=1)

                    cmap = 'coolwarm'
                    colors = color_map(x1[:-1], cmap)
                    line_segments = LineCollection(segments, colors=colors, linewidths=1.5, linestyles='solid', cmap=cmap)
                    m.add_collection(line_segments)
                # m.set_xlim(np.min(data_mean[:,:,0])-0.1*abs(np.min(data_mean[:,:,0])), np.max(data_mean[:,:,0])+0.1*abs(np.max(data_mean[:,:,0])))
                # m.set_ylim(np.min(data_mean[:,:,1])-0.1*abs(np.min(data_mean[:,:,1])), np.max(data_mean[:,:,1])+0.1*abs(np.max(data_mean[:,:,1])))
                m.set_xlim(-11,10)
                m.set_ylim(-10,10)
                fig.subplots_adjust(bottom=0.108)

                #colorbar
                l = 0.7
                b = 0
                w = 0.2
                h = 0.02 
                rect = [l,b,w,h] 
                cbar_ax = fig.add_axes(rect) 
                cmap1 = copy.copy(mpl.cm.coolwarm)
                norm1 = mpl.colors.Normalize(vmin=0, vmax=int(19/30*13))
                im1 = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap1)
                cbar1 = plt.colorbar(
                    im1, orientation='horizontal',
                    ticks=np.linspace(0, int(19/30*13), 2),
                    label='Time After Odor Delivery (s)',cax=cbar_ax)
                if if_save:
                    mpl.rcParams['pdf.fonttype'] = 42
                    mpl.rcParams['ps.fonttype'] = 42
                    the_save_path = figure_save_path +'/'+file_folder +'_No_shuffle'
                    f = os.path.exists(the_save_path)
                    if not f:
                        os.makedirs(the_save_path)
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename+'_tp_mean.png',dpi = 300,bbox_inches = 'tight')
                    plt.savefig(the_save_path + '/' +'channel_'+str(j_flag)+'_'+filename +'_tp_mean.pdf',dpi = 300,bbox_inches = 'tight')
                plt.close()
        j_flag = j_flag+1

['block_dim_c0.npy', 'channel_0_response_thresh_0.7_pca_thresh_0.9_block_size_0.1_odor_choice_0_downsample', 'channel_1_response_thresh_0.7_pca_thresh_0.9_block_size_0.1_odor_choice_0_downsample', 'channel_2_response_thresh_0.7_pca_thresh_0.9_block_size_0.1_odor_choice_0_downsample', 'list_list_accuracy.npy', 'list_list_accuracy_svm.npy', 'list_list_AUC_weighted.npy', 'list_list_f1_weighted.npy', 'list_list_num_dim.npy', 'list_list_stim.npy', 'p1-classification-pca-whole-brain-integration_region_lda_tp_pca_only_stim.ipynb', 'pca_data_c0.npy']
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
(180,)
['block_dim_c0.npy', 'channel_0_response_thresh_0.7_pca_thresh_0.9_block_size_0.1_odor_choice_0_downsample', 'channel_1_response_thresh_0.7_pca_thresh_0.9_block_size_0.1_odor_choice_0_downsample', 'channel_2_response_thresh_0.7_pca_thresh_0.